# Modelling With No Parameter Data

For this notebook I'll be looking at models with the same processing as the previous models, but without the use of the parameter data that was used to generate the samples.

In [1]:
import pickle
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import GridSearchCV

import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import accuracy_score

In [2]:
# code to toggle how much of the columns and rows that can be seen

pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None) 
# pd.reset_option('display.max_rows')
# pd.reset_option('display.max_columns')

In [3]:
# importing prepared dataset

data = pd.read_csv('../data/csvs/exercise_df_pre_modelling.csv')

In [4]:
# Dropping the parameter columns

columns_to_drop = ['watch_location', 'crown_orientation', 'ref_xy_rotation', 'rel_baseline_speed', 'max_rel_speed_change', 'wrist_offset_deg', 'avg_rep_time' ]
euler = data.drop(columns = columns_to_drop, axis=1)

In [5]:
euler

,exercise,euler_x_1,euler_y_1,euler_z_1,euler_x_2,euler_y_2,euler_z_2,euler_x_3,euler_y_3,euler_z_3,euler_x_4,euler_y_4,euler_z_4,euler_x_5,euler_y_5,euler_z_5,euler_x_6,euler_y_6,euler_z_6,euler_x_7,euler_y_7,euler_z_7,euler_x_8,euler_y_8,euler_z_8,euler_x_9,euler_y_9,euler_z_9,euler_x_10,euler_y_10,euler_z_10,euler_x_11,euler_y_11,euler_z_11,euler_x_12,euler_y_12,euler_z_12,euler_x_13,euler_y_13,euler_z_13,euler_x_14,euler_y_14,euler_z_14,euler_x_15,euler_y_15,euler_z_15,euler_x_16,euler_y_16,euler_z_16,euler_x_17,euler_y_17,euler_z_17,euler_x_18,euler_y_18,euler_z_18,euler_x_19,euler_y_19,euler_z_19,euler_x_20,euler_y_20,euler_z_20,euler_x_21,euler_y_21,euler_z_21,euler_x_22,euler_y_22,euler_z_22,euler_x_23,euler_y_23,euler_z_23,euler_x_24,euler_y_24,euler_z_24,euler_x_25,euler_y_25,euler_z_25,euler_x_26,euler_y_26,euler_z_26,euler_x_27,euler_y_27,euler_z_27,euler_x_28,euler_y_28,euler_z_28,euler_x_29,euler_y_29,euler_z_29,euler_x_30,euler_y_30,euler_z_30,euler_x_31,euler_y_31,euler_z_31,euler_x_32,euler_y_32,euler_z_32,euler_x_33,euler_y_33,euler_z_33,euler_x_34,euler_y_34,euler_z_34,euler_x_35,euler_y_35,euler_z_35,euler_x_36,euler_y_36,euler_z_36,euler_x_37,euler_y_37,euler_z_37,euler_x_38,euler_y_38,euler_z_38,euler_x_39,euler_y_39,euler_z_39,euler_x_40,euler_y_40,euler_z_40,euler_x_41,euler_y_41,euler_z_41,euler_x_42,euler_y_42,euler_z_42,euler_x_43,euler_y_43,euler_z_43,euler_x_44,euler_y_44,euler_z_44,euler_x_45,euler_y_45,euler_z_45,euler_x_46,euler_y_46,euler_z_46,euler_x_47,euler_y_47,euler_z_47,euler_x_48,euler_y_48,euler_z_48,euler_x_49,euler_y_49,euler_z_49,euler_x_50,euler_y_50,euler_z_50,euler_x_51,euler_y_51,euler_z_51,euler_x_52,euler_y_52,euler_z_52,euler_x_53,euler_y_53,euler_z_53,euler_x_54,euler_y_54,euler_z_54,euler_x_55,euler_y_55,euler_z_55,euler_x_56,euler_y_56,euler_z_56,euler_x_57,euler_y_57,euler_z_57,euler_x_58,euler_y_58,euler_z_58,euler_x_59,euler_y_59,euler_z_59,euler_x_60,euler_y_60,euler_z_60,euler_x_61,euler_y_61,euler_z_61,euler_x_62,euler_y_62,euler_z_62,euler_x_63,euler_y_63,euler_z_63,euler_x_64,euler_y_64,euler_z_64,euler_x_65,euler_y_65,euler_z_65,euler_x_66,euler_y_66,euler_z_66,euler_x_67,euler_y_67,euler_z_67,euler_x_68,euler_y_68,euler_z_68,euler_x_69,euler_y_69,euler_z_69,euler_x_70,euler_y_70,euler_z_70,euler_x_71,euler_y_71,euler_z_71,euler_x_72,euler_y_72,euler_z_72,euler_x_73,euler_y_73,euler_z_73,euler_x_74,euler_y_74,euler_z_74,euler_x_75,euler_y_75,euler_z_75,euler_x_76,euler_y_76,euler_z_76,euler_x_77,euler_y_77,euler_z_77,euler_x_78,euler_y_78,euler_z_78,euler_x_79,euler_y_79,euler_z_79,euler_x_80,euler_y_80,euler_z_80,euler_x_81,euler_y_81,euler_z_81,euler_x_82,euler_y_82,euler_z_82,euler_x_83,euler_y_83,euler_z_83,euler_x_84,euler_y_84,euler_z_84,euler_x_85,euler_y_85,euler_z_85,euler_x_86,euler_y_86,euler_z_86,euler_x_87,euler_y_87,euler_z_87,euler_x_88,euler_y_88,euler_z_88,euler_x_89,euler_y_89,euler_z_89,euler_x_90,euler_y_90,euler_z_90,euler_x_91,euler_y_91,euler_z_91,euler_x_92,euler_y_92,euler_z_92,euler_x_93,euler_y_93,euler_z_93,euler_x_94,euler_y_94,euler_z_94,euler_x_95,euler_y_95,euler_z_95,euler_x_96,euler_y_96,euler_z_96,euler_x_97,euler_y_97,euler_z_97,euler_x_98,euler_y_98,euler_z_98,euler_x_99,euler_y_99,euler_z_99,euler_x_100,euler_y_100,euler_z_100,euler_x_101,euler_y_101,euler_z_101,euler_x_102,euler_y_102,euler_z_102,euler_x_103,euler_y_103,euler_z_103,euler_x_104,euler_y_104,euler_z_104,euler_x_105,euler_y_105,euler_z_105,euler_x_106,euler_y_106,euler_z_106,euler_x_107,euler_y_107,euler_z_107,euler_x_108,euler_y_108,euler_z_108,euler_x_109,euler_y_109,euler_z_109,euler_x_110,euler_y_110,euler_z_110,euler_x_111,euler_y_111,euler_z_111,euler_x_112,euler_y_112,euler_z_112,euler_x_113,euler_y_113,euler_z_113,euler_x_114,euler_y_114,euler_z_114,euler_x_115,euler_y_115,euler_z_115,euler_x_116,euler_y_116,euler_z_116,euler_x_117,euler_y_117,euler_z_117,euler_x_118,euler_y_118,euler_z_118,euler_x_119,euler_y_119,euler_z_119,euler_x_120,euler_y_120,euler_

I will need to encode the exercise column again, however with the removal of all the parameter columns, I don't think any scaling will be necessary as the euler angles are all within a common range, so we will leave that out.

In [6]:
# Encoding categorical columns and saving mapping model 

columns_to_encode = ['exercise']

label_mappings = {}
enc = LabelEncoder()

for column in columns_to_encode:
    enc.fit_transform(euler[column])
    
    euler[f'{column}'] = enc.transform(euler[column])
    label_mappings[column] = dict(zip(enc.classes_, enc.transform(enc.classes_)))

# Display the label mappings
for column, mapping in label_mappings.items():
    print(f"Column: {column}")
    for label, encoded_value in mapping.items():
        print(f"  {label} -> {encoded_value}")
    print()

Column: exercise
  ARM_RAISE-DUMBBELL -> 0
  BICEP_CURL-DUMBBELL -> 1
  BIRD_DOG -> 2
  BURPEE -> 3
  CRUNCHES -> 4
  HAMMER_CURL-DUMBBELL -> 5
  OVERHEAD_PRESS-DUMBBELL -> 6
  TRICEP_KICKBACK-BENT_OVER+SINGLE_ARM-DUMBBELL-LEFT -> 7
  TRICEP_KICKBACK-BENT_OVER+SINGLE_ARM-DUMBBELL-RIGHT -> 8
  V_UP -> 9



In [7]:
# Splitting

X=euler.drop(['exercise'], axis=1)
y=euler['exercise']

In [8]:
# Stratified splitter

stratified_splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

# Use the splitter to get indices for the train and test sets
for train_index, test_index in stratified_splitter.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

With having the most luck with the XGBoost model in our first round of modelling, I am going to try that one first this time.

### XGBoost

In [9]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning, module="xgboost")
warnings.filterwarnings("ignore", category=UserWarning, message="`eval_metric` in `fit` method is deprecated*")
warnings.filterwarnings("ignore", category=UserWarning, message="`early_stopping_rounds` in `fit` method is deprecated*")

In [10]:
# parameter set up
params = {
    'objective': 'multi:softmax',
    'num_class': len(y_train.unique()),
    'max_depth': 6,
    'eta': 0.3,
    'subsample':1,
    'colsample_bytree':1
}

n_rounds = 100

In [11]:
# Convert to DMatrix

d_train = xgb.DMatrix(X_train, label=y_train)
d_test = xgb.DMatrix(X_test, label=y_test)

In [12]:
# Setting up the XGBoost model

model_xgb = xgb.train(params, d_train, n_rounds)

In [13]:
y_pred_xgb = model_xgb.predict(d_test)

In [14]:
# Checking accuracy score.

acc = accuracy_score(y_test, y_pred_xgb)
acc

0.9067796610169492

#### XGB with GridSearch

In [15]:
# Define the hyperparameter grid

param_grid = {
    'objective': ['multi:softmax'],
    'max_depth': [3, 4, 5, 6, 7],
    'eta': [0.2, 0.1, 0.01, 0.001],
    'subsample': [0.5, 0.7, 1],
}

# Train the initial XGBoost model
model_xgbc = XGBClassifier()
grid_search_xgb = GridSearchCV(model_xgbc, param_grid, cv=5, scoring='accuracy')

# Fit the grid search to the data
grid_search_xgb.fit(X_train, y_train)

print("Best set of hyperparameters: ", grid_search_xgb.best_params_)

Best set of hyperparameters:  {'eta': 0.1, 'max_depth': 7, 'objective': 'multi:softmax', 'subsample': 0.7}


In [16]:
# Setting up model with tuned parameters

best_params = grid_search_xgb.best_params_
best_xgb_model = xgb.XGBClassifier(**best_params)

best_xgb_model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eta=0.1, eval_metric=None,
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=7,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, ...)

In [17]:
# prediction

y_pred = best_xgb_model.predict(X_test)

In [18]:
# obtaining accuracy score

accuracy = accuracy_score(y_test, y_pred)
print("Test Accuracy: ", accuracy)

Test Accuracy:  0.9180790960451978


Almost 92% with some tuning and none of the parameter data is substantially better than I was expecting!

### RandomForest

In [19]:
# setting up basic model

rfc = RandomForestClassifier(random_state = 3)

In [20]:
# fitting the model to the stratified/scaled data

rfc.fit(X_train, y_train)

RandomForestClassifier(random_state=3)

In [21]:
# running the predictions on training and testing data

y_train_pred = rfc.predict(X_train)
y_test_pred = rfc.predict(X_test)

In [22]:
# accuracy results for basic RandomForestClassifier

print('Testing accuracy :' ,accuracy_score(y_test, y_test_pred))

Testing accuracy : 0.8813559322033898


How well both models did with the parameter values being removed is a bit surprising to me. I really expected those numbers to have more of an effect.

#### RFC with GridSearch

In [23]:
# Parameter settings

param_grid = {
    'n_estimators': [10, 50, 100],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2', None]
}

In [24]:
# Using GridSearchCV

grid_search_rfc = GridSearchCV(rfc, param_grid, cv=5, n_jobs=-1)
grid_search_rfc.fit(X_train, y_train)

print("Best Hyperparameters:", grid_search_rfc.best_params_)

best_model = grid_search_rfc.best_estimator_


Best Hyperparameters: {'max_depth': None, 'max_features': 'log2', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}


In [25]:
y_train_pred = best_model.predict(X_train)
y_pred_best_model = best_model.predict(X_test)

In [26]:
print('Train Accuracy score :', accuracy_score(y_train, y_train_pred))
print('Test Accuracy score :', accuracy_score(y_test, y_pred_best_model))

Train Accuracy score : 1.0
Test Accuracy score : 0.884180790960452


Tuning helped a tiny bit, but not enough to spend more time on. 88% accuracy with omitting the parameters is surprising to me. 

Next notebook: modelling_by_rep